# Decision Trees from Scratch

# Decision Trees from Scratch

```python
from collections import Counter
import numpy as np

class DecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.tree = None

    def fit(self, X, y):
        self.tree = self._build_tree(X, y)

    def _build_tree(self, X, y, depth=0):
        if len(set(y)) == 1:
            return y[0]  # If all labels are the same, return the label
        
        if self.max_depth and depth >= self.max_depth:
            return Counter(y).most_common(1)[0][0]  # Majority class

        best_split = self._find_best_split(X, y)
        left_tree = self._build_tree(*best_split['left'], depth + 1)
        right_tree = self._build_tree(*best_split['right'], depth + 1)
        
        return {'feature': best_split['feature'], 'threshold': best_split['threshold'],
                'left': left_tree, 'right': right_tree}

    def _find_best_split(self, X, y):
        best_split = {}
        min_gini = float('inf')
        
        for feature in range(X.shape[1]):
            thresholds = set(X[:, feature])
            for threshold in thresholds:
                left_mask = X[:, feature] <= threshold
                right_mask = ~left_mask
                left_y = y[left_mask]
                right_y = y[right_mask]
                
                gini = self._calculate_gini(left_y, right_y)
                if gini < min_gini:
                    min_gini = gini
                    best_split = {'feature': feature, 'threshold': threshold, 'left': (X[left_mask], left_y), 'right': (X[right_mask], right_y)}
        
        return best_split

    def _calculate_gini(self, left_y, right_y):
        left_size = len(left_y)
        right_size = len(right_y)
        total_size = left_size + right_size
        
        left_gini = 1 - sum((np.sum(left_y == label) / left_size) ** 2 for label in set(left_y))
        right_gini = 1 - sum((np.sum(right_y == label) / right_size) ** 2 for label in set(right_y))
        
        return (left_size / total_size) * left_gini + (right_size / total_size) * right_gini

    def predict(self, X):
        return np.array([self._predict_one(x, self.tree) for x in X])

    def _predict_one(self, x, tree):
        if isinstance(tree, dict):
            if x[tree['feature']] <= tree['threshold']:
                return self._predict_one(x, tree['left'])
            else:
                return self._predict_one(x, tree['right'])
        else:
            return tree
```

You can use this class to create a decision tree model and train it on any dataset, such as the Iris dataset, as follows:

```python
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load dataset
data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=42)

# Initialize Decision Tree model
model = DecisionTree(max_depth=5)

# Fit the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
```

